# AG2: Swarm (Concepts Code)

Concepts Code
- https://docs.ag2.ai/docs/user-guide/advanced-concepts/swarm/concept-code

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## 1st example
**demonstrates** (see matching numbers in the code):
1. How to update and return context variables in functions
2. How to transfer to another agent in a function
3. Combination of 1 and 2
4. A standard function
5. How handoffs are a convenient alternative to functions for transferring

In [2]:
import autogen
import random

from autogen import (
    AfterWork,
    OnCondition,
    AfterWorkOption,
    AssistantAgent,
    SwarmResult,
    initiate_swarm_chat,
    register_hand_off,
    # LLMConfig,
)

# llm_config = LLMConfig.from_json(path="<path_to_your_config_file>")
llm_config = {"api_type": "openai", "model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}

In [3]:
# 1. A function that returns a value of "success" and updates the context variable "1" to True
def update_context_1(context_variables: dict) -> SwarmResult:
    context_variables["1"] = True
    return SwarmResult(value="success", context_variables=context_variables)

In [4]:
# 2. A function that returns an AssistantAgent object
def transfer_to_agent_2() -> AssistantAgent:
    """Transfer to agent 2"""
    return agent_2

In [5]:
# 3. A function that returns the value of "success", updates the context variable and transfers to agent 3
def update_context_2_and_transfer_to_3(context_variables: dict) -> SwarmResult:
    context_variables["2"] = True
    return SwarmResult(value="success", context_variables=context_variables, agent=agent_3)

In [6]:
# 4. A function that returns a normal value
def get_random_number() -> str:
    return random.randint(1, 100)

In [8]:
def update_context_3_with_random_number(context_variables: dict, random_number: int) -> SwarmResult:
    context_variables["3"] = random_number
    return SwarmResult(value="success", context_variables=context_variables)

In [9]:
agent_1 = AssistantAgent(
    name="Agent_1",
    system_message="You are Agent 1, first, call the function to update context 1, and transfer to Agent 2",
    llm_config=llm_config,
    functions=[update_context_1, transfer_to_agent_2],
)

agent_2 = AssistantAgent(
    name="Agent_2",
    system_message="You are Agent 2, call the function that updates context 2 and transfer to Agent 3",
    llm_config=llm_config,
    functions=[update_context_2_and_transfer_to_3],
)

agent_3 = AssistantAgent(
    name="Agent_3",
    system_message="You are Agent 3, please transfer to Agent 4",
    llm_config=llm_config,
)

agent_4 = AssistantAgent(
    name="Agent_4",
    system_message="You are Agent 4, call the function to get a random number",
    llm_config=llm_config,
    functions=[get_random_number],
)

agent_5 = AssistantAgent(
    name="Agent_5",
    system_message="Update context 3 with the random number.",
    llm_config=llm_config,
    functions=[update_context_3_with_random_number],
)

In [10]:
# 5. This is equivalent to writing a transfer function
register_hand_off(agent=agent_3,hand_to=[OnCondition(agent_4, "Transfer to Agent 4")])
print("Agent 3's registered hand-offs:")
print(agent_3._swarm_conditional_functions)

Agent 3's registered hand-offs:
{'transfer_Agent_3_to_Agent_4': (<function register_hand_off.<locals>.make_transfer_function.<locals>.transfer_to_agent at 0x000001E4E183D620>, OnCondition(target=<autogen.agentchat.assistant_agent.AssistantAgent object at 0x000001E4E17E80D0>, condition='Transfer to Agent 4', available=None))}


In [11]:
# 6. When agent 4 replies without calling any functions, the `AfterWork` will be in effect, in this case, we transfer to agent 5
register_hand_off(agent=agent_4,hand_to=[AfterWork(agent_5)])

context_variables = {"1": False, "2": False, "3": False}
chat_result, context_variables, last_agent = initiate_swarm_chat(
    initial_agent=agent_1,
    agents=[agent_1, agent_2, agent_3, agent_4, agent_5],
    messages="start",
    context_variables=context_variables,
    after_work=AfterWork(AfterWorkOption.TERMINATE),  # this is the default value
)

_User (to chat_manager):

start

--------------------------------------------------------------------------------

Next speaker: Agent_1



C:\Users\Pavel\projects\ai-llm-agents\.venv\Lib\site-packages\pydantic\main.py:1630: RuntimeWarning: fields may not start with an underscore, ignoring "_x"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


Agent_1 (to chat_manager):

***** Suggested tool call (call_SuhJWf5PAz5aA3SYxmbVGEyZ): update_context_1 *****
Arguments: 
{}
*********************************************************************************
***** Suggested tool call (call_yimE1pbENGbaY6oin8fd5JQU): transfer_to_agent_2 *****
Arguments: 
{}
************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Swarm_Tool_Executor


>>>>>>>> EXECUTING FUNCTION update_context_1...
Call ID: call_SuhJWf5PAz5aA3SYxmbVGEyZ
Input arguments: {}

>>>>>>>> EXECUTING FUNCTION transfer_to_agent_2...
Call ID: call_yimE1pbENGbaY6oin8fd5JQU
Input arguments: {}
_Swarm_Tool_Executor (to chat_manager):

***** Response from calling tool (call_SuhJWf5PAz5aA3SYxmbVGEyZ) *****

**********************************************************************

--------------------------------------------------------------------------------
****

C:\Users\Pavel\projects\ai-llm-agents\.venv\Lib\site-packages\autogen\agentchat\conversable_agent.py:3002: UserWarning: Function 'transfer_Agent_3_to_Agent_4' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


Agent_3 (to chat_manager):

***** Suggested tool call (call_ENneOzQYAN48544LjMWJTCMW): transfer_Agent_3_to_Agent_4 *****
Arguments: 
{}
********************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Swarm_Tool_Executor


>>>>>>>> EXECUTING FUNCTION transfer_Agent_3_to_Agent_4...
Call ID: call_ENneOzQYAN48544LjMWJTCMW
Input arguments: {}
_Swarm_Tool_Executor (to chat_manager):

***** Response from calling tool (call_ENneOzQYAN48544LjMWJTCMW) *****
Swarm agent --> Agent_4
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: Agent_4

Agent_4 (to chat_manager):

***** Suggested tool call (call_2FUpjrYosK3QWvA2fOqEiK93): get_random_number *****
Arguments: 
{}
**********************************************************************************

----------

In [15]:
print(type(chat_result))
print(type(context_variables))
print(type(last_agent))

<class 'autogen.agentchat.chat.ChatResult'>
<class 'dict'>
<class 'autogen.agentchat.assistant_agent.AssistantAgent'>


In [16]:
print(len(chat_result.chat_history))
print(chat_result.summary)
print(chat_result.chat_history)

15
The context has been successfully updated with the random number **86**. What would you like to do next?
[{'content': 'start', 'role': 'assistant'}, {'content': 'None', 'tool_calls': [{'id': 'call_SuhJWf5PAz5aA3SYxmbVGEyZ', 'function': {'arguments': '{}', 'name': 'update_context_1'}, 'type': 'function'}, {'id': 'call_yimE1pbENGbaY6oin8fd5JQU', 'function': {'arguments': '{}', 'name': 'transfer_to_agent_2'}, 'type': 'function'}], 'name': 'Agent_1', 'role': 'assistant'}, {'content': '\nSwarm agent --> Agent_2', 'tool_responses': [{'tool_call_id': 'call_SuhJWf5PAz5aA3SYxmbVGEyZ', 'role': 'tool', 'content': ''}, {'tool_call_id': 'call_yimE1pbENGbaY6oin8fd5JQU', 'role': 'tool', 'content': 'Swarm agent --> Agent_2'}], 'name': '_Swarm_Tool_Executor', 'role': 'tool'}, {'content': 'None', 'tool_calls': [{'id': 'call_s8EHZmj7z7rM0gDpxW2u0XOT', 'function': {'arguments': '{}', 'name': 'update_context_2_and_transfer_to_3'}, 'type': 'function'}], 'name': 'Agent_2', 'role': 'assistant'}, {'content'

In [17]:
print(context_variables)

{'1': True, '2': True, '3': 86}


In [18]:
print(last_agent.name)

Agent_5


In [19]:
print(last_agent.system_message)

Update context 3 with the random number.


## 2nd example
**shows how to incorporate your own user agent into a swarm, allowing you to be a part of the swarm.**<br>
We pass in a UserProxyAgent to the swarm chat, through the user_agent parameter on initiate_swarm_chat, to accept user inputs. With agent_6, we register an AfterWork handoff to revert to the user agent when no tool calls are suggested.

In [20]:
from autogen import UserProxyAgent

user_agent = UserProxyAgent(name="User", code_execution_config=False)

agent_6 = AssistantAgent(
    name="Agent_6",
    system_message="You are Agent 6. Your job is to tell jokes.",
    llm_config=llm_config,
)

agent_7 = AssistantAgent(
    name="Agent_7",
    system_message="You are Agent 7, explain the joke.",
    llm_config=llm_config,
)

register_hand_off(
    agent=agent_6,
    hand_to=[
        OnCondition(
        agent_7, "Used to transfer to Agent 7. Don't call this function, unless the user explicitly tells you to."
        ),
        AfterWork(AfterWorkOption.REVERT_TO_USER),
    ]
)

In [21]:
# chat_result, _, _ = initiate_swarm_chat(
chat_result, context_variables, last_agent = initiate_swarm_chat(
    initial_agent=agent_6,
    agents=[agent_6, agent_7],
    user_agent=user_agent,
    messages="start",
)

User (to chat_manager):

start

--------------------------------------------------------------------------------

Next speaker: Agent_6

Agent_6 (to chat_manager):

Why did the bicycle fall over? 

Because it was two-tired! 

What's next on your mind?

--------------------------------------------------------------------------------

Next speaker: User



Replying as User. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [22]:
print(type(chat_result))
print(type(context_variables))
print(type(last_agent))

<class 'autogen.agentchat.chat.ChatResult'>
<class 'NoneType'>
<class 'autogen.agentchat.assistant_agent.AssistantAgent'>


In [23]:
print(len(chat_result.chat_history))
print(chat_result.summary)
print(chat_result.chat_history)

2
Why did the bicycle fall over? 

Because it was two-tired! 

What's next on your mind?
[{'content': 'start', 'role': 'assistant', 'name': 'User'}, {'content': "Why did the bicycle fall over? \n\nBecause it was two-tired! \n\nWhat's next on your mind?", 'name': 'Agent_6', 'role': 'user'}]


In [24]:
print(context_variables)

None


In [25]:
print(last_agent.name)

Agent_6


In [26]:
print(last_agent.system_message)

You are Agent 6. Your job is to tell jokes.
